In [ ]:
from google.colab import auth
auth.authenticate_service_account()

Successfully saved credentials for gsutil@vietai-research.iam.gserviceaccount.com


In [ ]:
%%capture
!pip install t5

In [1]:
!gsutil cp gs://scifive/finetune/chemprot/*_blurb.tsv .

Copying gs://scifive/finetune/chemprot/dev_blurb.tsv...
Copying gs://scifive/finetune/chemprot/test_blurb.tsv...
Copying gs://scifive/finetune/chemprot/train_blurb.tsv...
- [3 files][ 10.4 MiB/ 10.4 MiB]                                                
Operation completed over 3 objects/10.4 MiB.                                     


In [ ]:
import tensorflow
import functools
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import argparse
import tensorflow.compat.v1 as tf
import t5
from t5.models import MtfModel
import os
print(tensorflow.__version__)

2.9.2


In [ ]:

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
TPU_ADDRESS = tpu.get_master()
tf.disable_v2_behavior()
TPU_TOPOLOGY = 'v2-8'


print(f"TPU Address {TPU_ADDRESS}")
ON_CLOUD = True

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)


## Bypass an error
tf.compat.v1.flags.DEFINE_string('f','','')


TPU Address None


In [ ]:

task = 'chemprot'
def dumping_dataset(split, shuffle_files = False):
    del shuffle_files
    if split == 'train':
      ds = tf.data.TextLineDataset(
            [
            'gs://scifive/finetune/chemprot/train_blurb.tsv',
            ]
          )
    else:
      ds = tf.data.TextLineDataset(
            [
                        ]
          )
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                          field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.map(lambda *ex: dict(zip(["input", "target"], ex)))
    return ds

def preprocessor(ds):
  def to_inputs_and_targets(ex):
    """Map {"inputs": ..., "targets": ...}->{"inputs": {task}..., "targets": ...}."""
    return {
        "inputs":
            tf.strings.join(
                [f"{task}: ", ex["input"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove(task)
t5.data.TaskRegistry.add(
    task,
    dataset_fn=dumping_dataset,
    splits=["train", "validation"],
    text_preprocessor=[preprocessor],
    metric_fns=[],
)

t5.data.MixtureRegistry.remove("all")
t5.data.MixtureRegistry.add(
    "all",
    [task],
     default_rate=1.0,
)

INFO:root:system_path_file_exists:gs://scifive/models/pubmed/base/operative_config.gin
ERROR:root:Path not found: gs://scifive/models/pubmed/base/operative_config.gin
INFO:root:Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).
eval_on_tpu ignored because use_tpu is False.
From /usr/local/lib/python3.8/dist-packages/tensorflow/python/training/training_util.py:396: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
From /usr/local/lib/python3.8/dist-packages/seqio/dataset_providers.py:1537: sample_from_datasets_v2 (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.sample_from_datasets(...)`.
Using default tf glorot_unif

In [ ]:
MODEL_SIZE = 'base'

# Set parallelism and batch size to fit on v3-8 TPU (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (4, 256, 8),
    "large": (8, 256, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

PRETRAINED_DIR = f"gs://scifive/models/pubmed/{MODEL_SIZE}/"


# change to your own MODEL_DIR 
MODEL_DIR = "gs://YOURMODEL_/tmp/scifive_pubmed_base_chemprot_blurb"

model = MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 256, "targets": 8},
    learning_rate_schedule=0.001,
    iterations_per_loop=100,
    save_checkpoints_steps=2000,
    keep_checkpoint_max=5,
)

FINETUNE_STEPS = 25000
model.finetune(
   mixture_or_task_name="all",
   pretrained_model_dir=PRETRAINED_DIR,
   finetune_steps=FINETUNE_STEPS
)

In [ ]:
predicted_file = "test" or "dev"

print("======starting predicting=======")
with open(f"{predicted_file}_blurb.tsv") as file:
  with open("predict_input.txt", "w") as outfile:
    with open("actual_output.txt", "w") as outfile1:    
      for line in file:
        line = line.strip().split('\t')
        outfile.write(f"{task}: {line[0]}\n")
        outfile1.write(f"{line[1]}\n")

input_file = "predict_input.txt"
output_file = "predict_output.txt"

checkpoints = [int(x.replace('.index', '').split('-')[-1]) for x in tf.io.gfile.glob(MODEL_DIR +'/*ckpt*.index')]
checkpoints.sort()
checkpoints = checkpoints[-5:]
for checkpoint in checkpoints:
    if os.path.exists(f'{output_file}-{checkpoint}'):
        print(f"skipping {checkpoint}")
        continue
    with tf_verbosity_level('ERROR'):
      model.batch_size = 8 
      model.predict(
          input_file=input_file,
          output_file=output_file,
          checkpoint_steps=checkpoint,
          temperature=0,
    )

In [ ]:
##Evaluations script similiar to https://github.com/michiyasunaga/LinkBERT

TP = 0
P_total = 0
L_total = 0
for p,a in zip(predictions, references):
    p = p.strip()
    a = a.strip()
    if p == a and p != "0":
        TP += 1
    if p != "0":
        P_total += 1
    if a != "0":
        L_total += 1
P = TP / P_total if P_total else 0
R = TP / L_total if L_total else 0
F1 = 2 * P * R / (P + R) if (P + R) else 0
F1